In [1]:
import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.special import comb
from scipy.stats import hypergeom
from matplotlib import rcParams
from collections import defaultdict
rcParams['font.family'] = 'sans-serif'
rcParams['font.sans-serif'] = ['Arial']
rcParams['pdf.fonttype'] = 42
rcParams['axes.formatter.useoffset'] = False


Bad key "text.kerning_factor" on line 4 in
/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
http://github.com/matplotlib/matplotlib/blob/master/matplotlibrc.template
or from the matplotlib source distribution


In [2]:
def formatAxes(ax,text_size=10,xlim=None,xlabel=None,xticks=None,xticklabels=None,ylim=None,yticks=None,ylabel=None,yticklabels=None):
    """ to tweak the plotting of the axes as well as the fontsize """
    for loc,spine in list(ax.spines.items()):
        if loc == 'left': # settings for the y-axis
            if yticklabels and not yticks:
                sys.stderr.write("Need to provide a list wiht both y-labels and y-ticks!")
            if yticks:
                ax.yaxis.set_ticks_position('left')
                ax.yaxis.set_tick_params(direction='out')
                spine.set_visible(True)
                spine.set_position(("outward",1))
                spine.set_smart_bounds(True)
                ax.set_yticks(yticks)
                if ylim:
                    ax.set_ylim(ylim)
                if yticklabels:
                    ax.set_yticklabels(yticklabels,fontsize=text_size)
            else:
                spine.set_visible(False)
                ax.set_yticklabels([])
                ax.tick_params(axis='y',which='both',length=0)
                
            if ylabel:
                ax.set_ylabel(ylabel,fontsize=text_size,rotation=0,labelpad=160)
                ax.get_yaxis().set_label_coords(-0.1,0.5)
        elif loc == 'bottom': # settings for x-axis
            if xticks:
                spine.set_position('zero')
                spine.set_visible(False)
                spine.set_smart_bounds(False)
                ax.set_xticks(xticks)
                ax.tick_params(axis='x',which='both',length=0)
                if xlim:
                    ax.set_xlim(xlim)
                if xticklabels:
                    ax.set_xticklabels(xticklabels,fontsize=text_size)
            else:
                spine.set_visible(False)
                ax.set_xticklabels([])
                ax.tick_params(axis='x',which='both',length=0)
                
            if xlabel:
                ax.tick_params(axis='x',which='both',length=0)
                ax.set_xlabel(xlabel,fontsize=text_size+2)
                ax.xaxis.labelpad = 10
        else:
            spine.set_visible(False)
    ax.patch.set_visible(False)

### Loading the big dataframe:

In [3]:
alldata = pd.read_csv('../../../New_data_table.txt',\
                      sep="\t",\
                      header=0,\
                      index_col=0)

In [4]:
alldata.head()

,nucleotide,CELF1,FUS,HuR,PTBP1,RBFOX2,TARDBP,SHAPE_reactivity_ex_vivo_1,SHAPE_reactivity_ex_vivo_2,SHAPE_reactivity_in_cell_1,...,dStruct,dStruct_scaled,unmodified,ex_vivo,in_vivo,both,scaled_unmodified,scaled_ex_vivo,scaled_in_vivo,scaled_both
1,C,0,0,0,0,0,0,-999.0,-999.0,-999.0,...,0,0,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000
2,G,0,0,0,0,0,0,-999.0,-999.0,-999.0,...,0,0,0.712398,0.131639,0.131639,0.024325,0.712398,0.131639,0.131639,0.024325
3,G,0,0,0,0,0,0,-999.0,-999.0,-999.0,...,0,0,0.761379,0.111191,0.111191,0.016238,0.761379,0.111191,0.111191,0.016238
4,C,0,0,0,0,0,0,-999.0,-999.0,-999.0,...,0,0,0.795446,0.096432,0.096432,0.011690,0.795446,0.096432,0.096432,0.011690
5,T,0,0,0,0,0,0,-999.0,-999.0,-999.0,...,0,0,0.819621,0.085708,0.085708,0.008963,0.819621,0.085708,0.085708,0.008963


### Masking positions not considered by deltaSHAPE:

### ROC 

In [5]:
alldata.columns

Index(['nucleotide', 'CELF1', 'FUS', 'HuR', 'PTBP1', 'RBFOX2', 'TARDBP',
       'SHAPE_reactivity_ex_vivo_1', 'SHAPE_reactivity_ex_vivo_2',
       'SHAPE_reactivity_in_cell_1', 'SHAPE_reactivity_in_cell_2',
       'deltaSHAPE_rep1', 'deltaSHAPE_rep2', 'dStruct', 'dStruct_scaled',
       'unmodified', 'ex_vivo', 'in_vivo', 'both', 'scaled_unmodified',
       'scaled_ex_vivo', 'scaled_in_vivo', 'scaled_both'],
      dtype='object')

In [6]:
alldata.columns[1:7]

Index(['CELF1', 'FUS', 'HuR', 'PTBP1', 'RBFOX2', 'TARDBP'], dtype='object')

In [7]:
alldata.loc[1:78]

,nucleotide,CELF1,FUS,HuR,PTBP1,RBFOX2,TARDBP,SHAPE_reactivity_ex_vivo_1,SHAPE_reactivity_ex_vivo_2,SHAPE_reactivity_in_cell_1,...,dStruct,dStruct_scaled,unmodified,ex_vivo,in_vivo,both,scaled_unmodified,scaled_ex_vivo,scaled_in_vivo,scaled_both
1,C,0,0,0,0,0,0,-999.000000,-999.0,-999.000000,...,0,0,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000
2,G,0,0,0,0,0,0,-999.000000,-999.0,-999.000000,...,0,0,0.712398,0.131639,0.131639,0.024325,0.712398,0.131639,0.131639,0.024325
3,G,0,0,0,0,0,0,-999.000000,-999.0,-999.000000,...,0,0,0.761379,0.111191,0.111191,0.016238,0.761379,0.111191,0.111191,0.016238
4,C,0,0,0,0,0,0,-999.000000,-999.0,-999.000000,...,0,0,0.795446,0.096432,0.096432,0.011690,0.795446,0.096432,0.096432,0.011690
5,T,0,0,0,0,0,0,-999.000000,-999.0,-999.000000,...,0,0,0.819621,0.085708,0.085708,0.008963,0.819621,0.085708,0.085708,0.008963
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74,G,0,0,0,0,0,0,0.249767,-999.0,0.000000,...,0,0,0.902883,0.047319,0.047319,0.002480,0.902883,0.047319,0.047319,0.002480
75,C,0,0,0,0,0,0,-999.000000,-999.0,-999.000000,...,0,0,0.908434,0.044684,0.044684,0.002198,0.908434,0.044684,0.044684,0.002198
76,T,0,0,0,0,0,0,-999.000000,-999.0,-999.000000,...,0,0,0.915732,0.041207,0.041207,0.001854,0.915732,0.041207,0.041207,0.001854
77,T,0,0,0,0,0,0,0.126209,-999.0,1.176487,...,0,0,0.925281,0.036634,0.036634,0.001450,0.925281,0.036634,0.036634,0.001450


In [8]:
#SETTING NUCLEOTIDES TO ZERO IF FALLING IN REGIONS NOT USED BY DIFFBUM HMM
proteins = ["CELF1","FUS","HuR","PTBP1","RBFOX2","TARDBP"]


#for index, row in alldata.iterrows():
#    if index>=1 and index<=78 or index>=2451 and index<=2599 or index>=2500 and index<=4500 or index>=17801 and index<=17918:
#        for protein in proteins:
#            row[protein]=0

### This can be done more efficient: see below.
### This isn't the correct way to do it as you will add more false negatives.
### You have to remove the rows from the dataframe!

In [9]:
### The nice thing about pandas is that you don't have to do for loops to iterate over each row.
### Shorter and faster:

alldata.loc[1:78,proteins] = 0
alldata.loc[2451:2599,proteins] = 0
alldata.loc[2500:4500,proteins] = 0
alldata.loc[17801:17918,proteins] = 0

### We need to delete these rows:
first make a list of all the indices that you want to have deleted:

In [10]:
allpositions = list()
starts = [1,2451,2500,17801]
ends = [79,2600,4501,17919]
for start,end in list(zip(starts,ends)):
    allpositions.extend(np.arange(start,end))


In [11]:
allpositions[:10]

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

### Now deleting all the positions:

In [12]:
alldata.drop(allpositions,inplace=True)

In [13]:
alldata.head()

,nucleotide,CELF1,FUS,HuR,PTBP1,RBFOX2,TARDBP,SHAPE_reactivity_ex_vivo_1,SHAPE_reactivity_ex_vivo_2,SHAPE_reactivity_in_cell_1,...,dStruct,dStruct_scaled,unmodified,ex_vivo,in_vivo,both,scaled_unmodified,scaled_ex_vivo,scaled_in_vivo,scaled_both
79,G,0,0,0,0,0,0,0.031491,-0.128846,0.859859,...,0,0,0.953725,2.286328e-02,2.286328e-02,5.480920e-04,0.953725,2.286328e-02,2.286329e-02,5.480930e-04
80,G,0,0,0,0,0,0,-0.304057,-0.455325,-0.453171,...,0,0,0.974195,1.281835e-02,1.281835e-02,1.686630e-04,0.974195,1.281835e-02,1.281836e-02,1.686630e-04
81,G,0,0,0,0,0,0,-0.883056,-0.652497,-0.441984,...,0,0,1.000000,1.490000e-25,9.630000e-19,1.440000e-43,1.000000,6.060000e-21,5.730000e-15,3.470000e-35
82,G,0,0,0,0,0,0,-1.593320,-0.378628,-1.729645,...,0,0,0.994483,2.754800e-03,2.754800e-03,7.630000e-06,0.986340,6.823904e-03,6.789031e-03,4.700000e-05
83,G,0,0,0,0,0,0,-999.000000,-999.000000,-999.000000,...,0,0,1.000000,1.510000e-37,4.420000e-22,6.650000e-59,0.981774,9.110841e-03,9.031603e-03,8.380000e-05


In [45]:
#FOR ROC we need to plot true positive rate and false positive rate

#Used DC-SCALED diffBUM-HMM 


threshold=0.95
dict_false_positives={}
dict_true_positives={}
dict_false_negatives={}
dict_true_negatives={}

for protein in proteins:
    dict_false_positives[protein]= 0
    dict_true_positives[protein]= 0
    dict_false_negatives[protein]= 0
    dict_true_negatives[protein]= 0

del protein


for protein in proteins:

    #for index, row in CELF1.iterrows():
    protein_actual_pos=alldata[alldata[protein] == 1]
    protein_actual_neg=alldata[alldata[protein] == 0]
    

    
    
    for index, row in protein_actual_pos.iterrows():

        #TP is a position that has binding site for a given protein  as indicated by CLIPD, 
        #and for which diffBUM-HMM predicts a differential reactive nucleotide either ex vivo or in vivo
        if row['scaled_ex_vivo'] >= threshold or row['scaled_in_vivo'] >= threshold:
            dict_true_positives[protein]= dict_true_positives[protein]+1

        #FN is a position that has binding site for a given protein as indicated by CLIPD,
        #and for which diffBUM-HMM does NOT predict a differential reactive nucleotide either ex vivo or in vivo
        else:
            dict_false_negatives[protein]= dict_false_negatives[protein]+1

    for index, row in protein_actual_neg.iterrows():
        
      
    
        if row['scaled_ex_vivo'] >= threshold or row['scaled_in_vivo'] >= threshold:
             #FP is a position that has NO binding site for a given protein as indicated by CLIPD,
     #and for which diffBUM-HMM predicts a differential reactive nucleotide either ex vivo or in vivo 
            dict_false_positives[protein]= dict_false_positives[protein]+1
        
        else:
            #TN is a position that has NO binding site for a given protein as indicated by CLIPD,
     #and for which diffBUM-HMM does NOT predicts a differential reactive nucleotide either ex vivo or in vivo 
            dict_true_negatives[protein]= dict_true_negatives[protein]+1

print('TP' +str(dict_true_positives))
print('FN' +str(dict_false_negatives))
print('FP'+ str(dict_false_positives))
print('TN' + str(dict_true_negatives))

del protein

#TPR = TP/(TP+FN)
#FPR = FP/(FP+TN)

dict_false_positive_rate={}
dict_true_positive_rate={}

for protein in proteins:
    dict_false_positive_rate[protein]= 0
    dict_true_positive_rate[protein]= 0

del protein

for protein in proteins:
    
    dict_true_positive_rate[protein]=dict_true_positives[protein]/(dict_true_positives[protein]+dict_false_negatives[protein])
    dict_false_positive_rate[protein]= dict_false_positives[protein]/(dict_false_positives[protein]+dict_true_negatives[protein])


print('TPR ' + str(dict_true_positive_rate))
print('FPR ' + str(dict_false_positive_rate))
   



TP{'CELF1': 77, 'FUS': 440, 'HuR': 129, 'PTBP1': 73, 'RBFOX2': 4, 'TARDBP': 68}
FN{'CELF1': 725, 'FUS': 2929, 'HuR': 1020, 'PTBP1': 531, 'RBFOX2': 40, 'TARDBP': 466}
FP{'CELF1': 1850, 'FUS': 1487, 'HuR': 1798, 'PTBP1': 1854, 'RBFOX2': 1923, 'TARDBP': 1859}
TN{'CELF1': 13020, 'FUS': 10816, 'HuR': 12725, 'PTBP1': 13214, 'RBFOX2': 13705, 'TARDBP': 13279}
TPR {'CELF1': 0.09600997506234414, 'FUS': 0.13060255268625706, 'HuR': 0.1122715404699739, 'PTBP1': 0.12086092715231789, 'RBFOX2': 0.09090909090909091, 'TARDBP': 0.12734082397003746}
FPR {'CELF1': 0.12441156691324814, 'FUS': 0.12086482971632935, 'HuR': 0.12380362184121738, 'PTBP1': 0.12304220865410141, 'RBFOX2': 0.12304837471205529, 'TARDBP': 0.12280354075835645}


In [49]:
#FOR ROC we need to plot true positive rate and false positive rate

#Used DC-SCALED diffBUM-HMM 

threshold = 0.95

proteins = ["CELF1","FUS","HuR","PTBP1","RBFOX2","TARDBP"]

#TP is a position that has binding site for any protein as indicated by CLIPD, 
#and for which diffBUM-HMM predicts a differential reactive nucleotide either ex vivo.

allbindingsites = alldata[alldata[proteins].any(axis=1)]

### TP is all those positions that have a protein binding site but also called diff modified:
allbindingsiteswithdiffmod = allbindingsites[allbindingsites["scaled_ex_vivo"] >= 0.95]
TP = len(allbindingsiteswithdiffmod.index)

#FN is a position that has binding site for any protein as indicated by CLIPD,
#and for which diffBUM-HMM does NOT predict a differential reactive nucleotide ex vivo.

allbindingsitesnodiffmod = allbindingsites[allbindingsites["scaled_ex_vivo"] < 0.95]
FN = len(allbindingsitesnodiffmod.index)

#FP is a position that has NO binding site any protein as indicated by CLIPD, 
#and for which diffBUM-HMM predicts a differential reactive nucleotide either ex vivo.

nobindingsites = alldata[alldata[proteins].sum(axis=1) == 0]
FP = len(nobindingsites[nobindingsites["scaled_ex_vivo"] >= 0.95].index)

#TN is a position that has NO binding site any protein as indicated by CLIPD, 
#and for which diffBUM-HMM does NOT predicts a differential reactive nucleotide ex vivo.

TN = len(nobindingsites[nobindingsites["scaled_ex_vivo"] < 0.95].index)

TPR = TP/(TP+FN)

FPR = FP/(FP+TN)

print("True positives:\t%s\nFalse negatives:\t%s\nFalse positive:\t%s\nTrue negative:\t%s\n\nTPR:\t%s\nFPR\t%s" % \
      (TP,FN,FP,TN,TPR,FPR))
  

True positives:	567
False negatives:	4576
False positive:	1172
True negative:	9357

TPR:	0.11024693758506708
FPR	0.1113116155380378
